In [ ]:
%pylab inline
import pysal as ps
import numpy as np

In [ ]:
ps.examples.explain('mexico')

In [ ]:
# we will use an updated shapefile rather than the built-in dataset

shp_link = 'data/mexicojoin.shp'
data_table = ps.pdio.read_files(shp_link)

In [ ]:
data_table.head()

In [ ]:
data_table.columns

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

shp = gpd.read_file(shp_link)

years = range(1940, 2010, 10)

for year in years:
    yi = ps.Quantiles(data_table['PCGDP%d'%year], k=5)
    f, ax = plt.subplots(1, figsize=(9, 9))
    shp.assign(cl=yi.yb).plot(column='cl', categorical=True, \
        k=5, cmap='BuGn', linewidth=0.1, ax=ax, \
        edgecolor='grey', legend=True)
    ax.set_axis_off()
    plt.title(str(year))
    plt.show()


In [ ]:
vnames= ["PCGDP%d"%year for year in years]
vnames

In [ ]:
Y = data_table[vnames].values

In [ ]:
Y.shape

## Theil Inequality

$$
T = \sum_{i=1}^n \left( \frac{y_i}{\sum_{i=1}^n y_i} ln \left[ n \frac{y_i}{\sum_{i=1}^n y_i} \right] \right)
$$

In [ ]:
theil = ps.Theil(Y)

In [ ]:
%pylab inline

In [ ]:
plot(years, theil.T)

## Interregional Decomposition

$$
T^t = \sum_{g=1}^{\omega} s_g^t ln(n/n_g s_g^t) + \sum_{g=1}^{\omega} s_g^t \sum_{i \in g} s_{i,g}^t ln (n_g s_{i,g}^t)
$$

where:

* $\sum_g n_g = n$
* $s_g^t = \sum_{i \in g} y_{i,g}^t / \sum_i^n y_i^t$
* $s_{i,g}^t = y_{i,g}^t / \sum_{i \in g} y_{i,g}^t$

$$
T^t = BG^t + WG^T
$$

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
shp = gpd.read_file(shp_link)


f, ax = plt.subplots(1, figsize=(9, 9))
shp.plot(column='HANSON98', categorical=True, \
       linewidth=0.1, ax=ax, \
        edgecolor='white', legend=True)
ax.set_axis_off()
plt.title('HANSON98')
plt.show()


In [ ]:
regimes = data_table['HANSON98'].values

In [ ]:
regimes

In [ ]:
theil_d = ps.TheilD(Y, regimes)

In [ ]:
regimes.shape

In [ ]:
Y.shape

In [ ]:
plot(years, theil_d.bg)

In [ ]:
theil_d.bg

In [ ]:
plot(years, theil_d.wg)

In [ ]:
plot(years, theil_d.T, label='T')
plot(years, theil_d.wg, '-o', label='wg')
plot(years, theil_d.bg, '-^', label='bg')
legend()

## Inference

In [ ]:
np.random.seed(12345)
theil_ds = ps.TheilDSim(Y, regimes, 999)

In [ ]:
theil_ds.bg_pvalue

In [ ]:
plot(years, theil_d.T, label='T')
plot(years, theil_d.wg, '-o', label='wg')
plot(years, theil_d.bg, '-^', label='bg')
plot(years, theil_ds.bg_pvalue, ':k', label='p-value')
legend()

## Sensitivity to regime definition

In [ ]:
regimes = ['HANSON03', 'HANSON98', 'ESQUIVEL99', 'INEGI', 'INEGI2']

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
shp = gpd.read_file(shp_link)

for regime in regimes:
    f, ax = plt.subplots(1, figsize=(9, 9))
    shp.plot(column=regime, categorical=True, \
           linewidth=0.1, ax=ax, \
            edgecolor='white', legend=True)
    ax.set_axis_off()
    plt.title(regime)
    plt.show()

In [ ]:
results = {}
for regime in regimes:
    reg = data_table[regime].values
    results[regime] = ps.TheilD(Y, reg)
    plot(years, results[regime].bg)

    

In [ ]:
results = {}
for regime in regimes:
    reg = data_table[regime].values
    results[regime] = ps.TheilD(Y, reg)
    plot(years, results[regime].bg, label=regime)

legend()

    

In [ ]:
#share of total equality
results = {}
for regime in regimes:
    reg = data_table[regime].values
    results[regime] = ps.TheilD(Y, reg)
    plot(years, results[regime].bg/theil.T, label=regime)

legend()

    